# Exact Inference in Bayesian Networks

Now that we have a Bayesian network, what can we do with it? Well, since the Bayesian network is simply a representation of the full joint probability distribution we can do anything we would have done with the joint distribution. Any quantity that could be calculated from the joint distribution can be computed from the Bayesian network.

The straightforward way to do this would simply be to construct the full joint distribution directly, but that would be to miss the point. One of the reasons we use Bayesian networks is to reduce the number of parameters needed to represent the joint distribution and so to construct the full joint distribution from the network would be both wasteful and would also render the efficient and meaning representation provided by the network redundant.

However, we don't need to do that, because we can compute anything we can compute from the joint distribution directly from the Bayesian network factorisation. Let's consider again the employment prospects of our school-leavers for which we had that the joint distribution is $P(G,M,F,P)$ which we write as `JointPDF`:

In [23]:
import numpy as np
JointPDF = np.zeros([2,2,2,2])
JointPDF[0][0][:][:] = np.array([[.066,0.054],[0.0165,0.0135]])
JointPDF[0][1][:][:] = np.array([[.081,.099],[.1215,.1485]])
JointPDF[1][0][:][:] = np.array([[.044,.036],[.011,.009]])
JointPDF[1][1][:][:] = np.array([[.054,.066],[.081,.099]])

We now know that this can be factorised as $P(G,M,F,P) = P(P\vert M)P(F\vert M)P(M)P(G)$. We'll compute the four terms directly from the `JointPDF` although in practice one would know these independently. 

In [24]:
P_G = JointPDF.sum(axis=(1,2,3))
P_M = JointPDF.sum(axis=(0,2,3))
P_F_M = np.array([JointPDF.sum(axis=(0,3))[0]/P_M[0],JointPDF.sum(axis=(0,3))[1]/P_M[1]])
P_P_M = np.array([JointPDF.sum(axis=(0,2))[0]/P_M[0],JointPDF.sum(axis=(0,2))[1]/P_M[1]])

print(P_G)
print(P_M)
print(P_F_M)
print(P_P_M)


[0.6 0.4]
[0.25 0.75]
[[0.8 0.2]
 [0.4 0.6]]
[[0.55 0.45]
 [0.45 0.55]]


Note the compactness of this representation: There are just six independent numbers here.

So how do we now derive other quantities from this? We sum up various terms just as we would with the joint distribution. Let consider an example. How would we calculate the distribution $P(F,P)$ from this. From the joint distribution this would be easy, we would do

In [25]:
P_FP = JointPDF.sum(axis=(0,1))
print(P_FP)

[[0.245 0.255]
 [0.23  0.27 ]]


It is somewhat more complex to do this from the factorised distribution, but only in practical terms; conceptually it is identical:

$P(F,P) = \sum_{M,G}P(G,M,F,P) = \sum_{M,G}P(P\vert M)P(F\vert M)P(M)P(G)$

We can simplify this calculation a bit by noting that $G$ is only in one term in the summand and so we can write

$P(F,P) = \sum_{M}P(P\vert M)P(F\vert M)P(M)\cdot\sum_G P(G)$

Noting further that $\sum_G P(G) = 1$ we have

$P(F,P) = \sum_{M}P(P\vert M)P(F\vert M)P(M)$.

Taking one term $P_(F=\mathrm{True},P=\mathrm{True}) = P(f,p)$ from the joint distribution as an example, we evaluate this as:
$P(f,p) = \sum_M P(p\vert M)P(f\vert M)P(M) = P(p\vert m)P(f\vert m)P(m) + P(p\vert \lnot m)P(f\vert \lnot m)P(\lnot m)$.

In [26]:
P_fm = P_F_M[0,0] * P_P_M[0,0] * P_M[0] + P_F_M[1,0] * P_P_M[1,0] * P_M[1] 

Notice that this does indeed correspond to the same quantity calculated directly from the joint distribution. Let's do another example. Can we calculate $P(M\vert P)$? Yes, sure.

$P(M\vert P) = \frac{P(M,P)}{P(P)} = \frac{\sum_{F,G}P(P\vert M)P(F\vert M)P(M)P(G)}{\sum_{F,M,G}P(P\vert M)P(F\vert M)P(M)P(G)}$.
We do not need to take this any further to realise that this will take an awful lot of work to get a result. Even the simple joint distribution we computed earlier was hard going: some algebra, plus four multiplications and an addition for one term of the joint distribution. We note that calculating directly from the joint distribution itself requires only a few addition although we would, of course first have to form the joint distribution from the factors which will require a significant number of multiplications. It is possible to generalise the algebraic aspects of this but the arithmetic itself remains.

For large problems with many variables this approach is unlikely to be tractable. It can be shown to scale as $O(N2^N)$ which is about as bad as it gets. There are approaches that can accelerate this by removing duplicate calculation but that will only get us to $O(2^N)$ which is still pretty bad. This is not approach we will use in practice for any but the simplest problems.

What is the alternative? It is to solve inference problems by *approximate sampling*. This will be our next topic.